In [13]:
import os
import struct
import sys
from array import array
from os import path


In [14]:
import torchvision

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=Trueraw

SyntaxError: incomplete input (246987043.py, line 4)

In [16]:
import struct
def read(dataset):
    if dataset is "training":
        path_img = "data/MNIST/raw/train-images-idx3-ubyte"
        path_lbl = "data/MNIST/raw/train-labels-idx1-ubyte"
        
    elif dataset is "testing":
        path_img = "data/MNIST/raw/t10k-images-idx3-ubyte"
        path_lbl = "data/MNIST/raw/t10k-labels-idx1-ubyte"
        
    else:
        raise ValueError("dataset must be 'training' or 'testing'")
    
    with open(path_lbl, 'rb') as f_labels:
        _, size = struct.unpack(">II", f_labels.read(8))
        lbl = array("B", f_labels.read())
        
    with open(path_img, 'rb') as f_img:
        _, size, rows, cols = struct.unpack(">IIII", f_img.read(16))
        img = array("B", f_img.read())
        
    return lbl, img, size, rows, cols

<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/w_/q1crhrb50bs6pn84kphn0qtw0000gn/T/ipykernel_4941/708247055.py:3: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if dataset is "training":
/var/folders/w_/q1crhrb50bs6pn84kphn0qtw0000gn/T/ipykernel_4941/708247055.py:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif dataset is "testing":


In [19]:
import numpy as np
from PIL import Image


def write_dataset(labels, data, size, rows, cols, output_dir):
    classes = {i: f"class_{i}" for  i in range(10)}
    
    output_dirs = [path.join(output_dir, classes[i]) for i in range(10)]
    
    for dir in output_dirs:
        if not path.exists(dir):
            os.makedirs(dir)
    
    for (i, label) in enumerate(labels):
        output_filename = path.join(output_dirs[label], str(i) + ".jpg")
        print("writing", output_filename)
        
        with open(output_filename, "wb") as h:
            data_i = [data[(i*rows*cols + j*cols) : (i*rows*cols + (j+1)*cols)] for j in range(rows)]
            data_array = np.asarray(data_i)
            
            im = Image.fromarray(data_array)
            im.save(output_filename)

In [20]:
output_path = ("data/mnist")

for dataset in ["training", "testing"]:
    write_dataset(*read(dataset), path.join(output_path, dataset))

writing data/mnist/training/class_5/0.jpg
writing data/mnist/training/class_0/1.jpg
writing data/mnist/training/class_4/2.jpg
writing data/mnist/training/class_1/3.jpg
writing data/mnist/training/class_9/4.jpg
writing data/mnist/training/class_2/5.jpg
writing data/mnist/training/class_1/6.jpg
writing data/mnist/training/class_3/7.jpg
writing data/mnist/training/class_1/8.jpg
writing data/mnist/training/class_4/9.jpg
writing data/mnist/training/class_3/10.jpg
writing data/mnist/training/class_5/11.jpg
writing data/mnist/training/class_3/12.jpg
writing data/mnist/training/class_6/13.jpg
writing data/mnist/training/class_1/14.jpg
writing data/mnist/training/class_7/15.jpg
writing data/mnist/training/class_2/16.jpg
writing data/mnist/training/class_8/17.jpg
writing data/mnist/training/class_6/18.jpg
writing data/mnist/training/class_9/19.jpg
writing data/mnist/training/class_4/20.jpg
writing data/mnist/training/class_0/21.jpg
writing data/mnist/training/class_9/22.jpg
writing data/mnist/tr

In [22]:
import json

if not os.path.exists("dataset"):
    os.mkdir("dataset")

img = np.random.randint(0, 50, [100000, 64, 64], dtype=np.uint8)
square = np.random.randint(100, 200, [100000, 15, 15], dtype=np.uint8)

coords = np.empty([100000, 2])

data = {}

for i in range(img.shape[0]):
    x = np.random.randint(20, 44)
    y = np.random.randint(20, 44)
    
    img[i, (y-7):(y+8), (x-7):(x+8)] = square[i]
    coords[i] = [y, x]
    
    name_img = f'img_{i}.jpg'
    path_img = os.path.join("dataset", name_img)
    
    image = Image.fromarray(img[i])
    image.save(path_img)
    
    data[name_img] = [y, x]
    
with open("dataset/coords.json", "w") as f:
    json.dump(data, f, indent=2)
    
    

### MNIST. Пользовательский класс данных

In [23]:
from torch.utils.data import Dataset, DataLoader


In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        
        self.len_dataset = 0
        self.data_list =  []
        
        for path_dir, dir_list, file_list in os.walk(path):
            if path_dir == path:
                self.classes = dir_list
                self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
                continue
                
            cls = path_dir.split('/')[-1]
            
            for name_file in file_list:
                file_path = os.path.join(path_dir, name_file)
                self.data_list.append((file_path, self.class_to_idx[cls]))
                
            self.len_dataset += len(file_list)

    def __len__(self):
        return self.len_dataset

    def __getitem__(self, index):
        file_path, target = self.data_list[index]
        sample = np.array(Image.open(file_path))
        
        if self.transform is not None:
            sample = self.transform(sample)
            
        return sample, target